In [88]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.linear_model import LinearRegression

In [89]:
params = pd.read_csv("../data/interim/interim_data.csv")
params.head()

,ID,StudySample,LaserPowerHatch,LaserSpeedHatch,HatchSpacing,LaserPowerContour,Porosity,LayerHeight,EnergyDensityCalculated,Machine,Powder,MicroCTScan,HatchOffsetFromCountour
0,1,Plessis_1-1,280.0,1200.0,0.14,150.0,0.013,0.03,55.555556,EOS M290,EOS-supplied Ti6Al4V ELI,NaN,0.015
1,2,Plessis_1-2,252.0,1200.0,0.14,135.0,0.008,0.03,50.000000,EOS M290,EOS-supplied Ti6Al4V ELI,NaN,0.015
2,3,Plessis_1-3,224.0,1200.0,0.14,120.0,0.007,0.03,44.444444,EOS M290,EOS-supplied Ti6Al4V ELI,NaN,0.015
3,4,Plessis_1-4,197.0,1200.0,0.14,105.0,0.045,0.03,39.087302,EOS M290,EOS-supplied Ti6Al4V ELI,NaN,0.015
4,5,Plessis_1-5,308.0,1200.0,0.14,165.0,0.017,0.03,61.111111,EOS M290,EOS-supplied Ti6Al4V ELI,NaN,0.015


In [90]:
machine_title = params.Machine[0]
alloy_title = params.Powder[0]

In [91]:
try:
    params = params.drop(["ID","StudySample","HatchOffsetFromCountour", "MicroCTScan", "LayerHeight", "Machine", "Powder"], axis=1)
    print("dropped columns")
except:
    print("already dropped columns")
params.columns = ["LaserPowerHatch","LaserSpeedHatch","HatchSpacing","LaserPowerContour","EnergyDensityCalculated","Porosity"]
params.head()

dropped columns


,LaserPowerHatch,LaserSpeedHatch,HatchSpacing,LaserPowerContour,EnergyDensityCalculated,Porosity
0,280.0,1200.0,0.14,150.0,0.013,55.555556
1,252.0,1200.0,0.14,135.0,0.008,50.000000
2,224.0,1200.0,0.14,120.0,0.007,44.444444
3,197.0,1200.0,0.14,105.0,0.045,39.087302
4,308.0,1200.0,0.14,165.0,0.017,61.111111


In [104]:
split_pct = .25
seed = 100

train_data = params.sample(frac=1-split_pct, random_state=seed).reset_index()
test_data = params.drop(train_data.index).sample(frac=1, random_state=seed).reset_index()
train_data.head()

,index,LaserPowerHatch,LaserSpeedHatch,HatchSpacing,LaserPowerContour,EnergyDensityCalculated,Porosity
0,6,364.0,1200.0,0.14,195.0,0.037,72.222222
1,19,280.0,800.0,0.14,150.0,0.179,83.333333
2,11,280.0,1200.0,0.17,150.0,0.026,45.751634
3,20,320.0,800.0,0.14,171.0,0.199,95.238095
4,17,200.0,800.0,0.14,107.0,0.036,59.523810


In [100]:
train_vals = train_data.values
test_vals = test_data.values
X = train_vals[:,1:-2]
y = train_vals[:,:-2:-1]
Xtest = test_vals[:,1:-2]
ytest = test_vals[:,:-2:-1]

model1 = LinearRegression()
model1.fit(X, y)

model2 = LinearRegression()
model2.fit(X, y.T[0])

print(model1.score(Xtest, ytest))
print(model2.score(Xtest, ytest))

0.9747616264918479
0.9747616264918479


In [103]:
X = train_vals[:,-2:-1]
X

array([[3.70e-02],
       [1.79e-01],
       [2.60e-02],
       [1.99e-01],
       [3.60e-02],
       [1.74e-01],
       [1.82e+01],
       [3.27e-01],
       [1.70e-02],
       [1.10e-02],
       [1.30e-02],
       [8.00e-03],
       [6.16e-01],
       [7.70e+00],
       [1.09e-01],
       [2.10e-02],
       [7.00e-03]])